In [1]:
import chess
import chess.engine
import os
import sys
import chess.pgn
import subprocess

current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)

import utils.chess_utils as chess_utils
from utils.pikeBot_chess_utils import *
from utils.utils import *
config=get_paths()
stockfish_path=config['stockfish_path']
maia_path=config['maia_path']
from evaluation_utils import *

In [2]:
def get_maia_path(maia_elo):
    version=int(str(maia_elo)[1])
    if version not in [1,5,9]:
        raise ValueError("Maia elo must be from (1100, 1500, 1900)")
    else:
        return maia_path.format(version=version)


In [3]:
class MaiaChess(chess_utils.Player):
    def __init__(self, maia_path: str) -> None:
        super().__init__()
        #redirect error stderr not to display messages from the engine
        self.engine = chess.engine.SimpleEngine.popen_uci(maia_path, stderr=subprocess.DEVNULL)

    def get_best_move(self, board):
        result = self.engine.play(board, chess.engine.Limit(nodes=1))
        move = result.move
        return move

### Parameters setup
set path parameters to your local resources and evaluation parameters depending on your evaluation setup

In [4]:
model_path = config['pikebot']
preprocessing_parameters_path = config['preprocessing_params_path']

#evaluation parameters
time_limit = 0.01
evaluation_dataset_paths = [
    'games/silversuite.pgn',
    ]

### Engine definition
By default engine1 is a maia chess engine simulating human behavior but you can modify it if you want other engine comparison. Both engines must be of a child class of chess_utils.Player.

In [5]:
maia_elo = 1100
maia_path = get_maia_path(maia_elo)
engine1 = MaiaChess(maia_path)

In [6]:
from Pikebot.PikeBotHeuristic5 import PikeBotHeuristic5
from Pikebot.PikeBotHeuristic6 import PikeBotHeuristic6
from utils.pikeBot_chess_utils import *
model = PikeBotModelWrapper(model_path, preprocessing_parameters_path)
engine2 = PikeBotHeuristic5(
    model,
    chess_utils.max_aggr,
    stockfish_path
)
engine3 = PikeBotHeuristic6(
    model,
    chess_utils.max_aggr,
    stockfish_path
)

### Dataset loading

In [7]:
boards_datasets = dict()
for path in evaluation_dataset_paths:
    boards_dataset = get_boards_dataset(path)

    boards_datasets[path] = boards_dataset


In [8]:
boards_datasets['games/silversuite.pgn'] = boards_datasets['games/silversuite.pgn'][:2]

### Evaluation code

In [ ]:
evaluate_engines(engine1, engine2, boards_datasets, maia_elo, 800)

In [ ]:
# evaluate_engines(engine1, engine3, boards_datasets, maia_elo, 800)

EngineError: expected value for option 'MultiPV' to be at least 1, got: 0